In [ ]:
# run()

In [ ]:
from agent.base import Agent
from optimization.Optimizer import PyTorchObjective

import matplotlib.pyplot as plt

# gym_gvgai.dir

%matplotlib inline
import gym
import gvgai
from IPython import display

In [ ]:
def show_state(env, step=0, name="", info=""):
    """Fn to visualize the agent playing the game in a notebook
    """
    plt.figure(10)
    plt.clf()
    plt.imshow(env.render(mode="rgb_array")[0])
    plt.title("{} | Step: {} {}".format(name, step, info))
    plt.axis('off')
    display.clear_output(wait=True)
    display.display(plt.gcf())

In [ ]:
# import numpy as np
# from agent.base import simulate
# from generator.levels.base import _initialize
# from generator.levels.base import Generator
# from generator.levels.CPPNGen import CPPNGen

In [ ]:
import os

In [ ]:
# x = CPPNGen(_initialize(os.path.join(gym_gvgai.dir + "/envs/games/zelda_v0/zelda_lvl2.txt")))

In [ ]:
# x.mutate(1)

In [ ]:
# sorted(os.listdir(os.path.join(gym_gvgai.dir, 'envs/games/zelda_v0/levels')))

In [ ]:
# for f in sorted(os.listdir(os.path.join(gym_gvgai.dir, 'envs/games/zelda_v0/levels'))):
#     os.remove(os.path.join(gym_gvgai.dir, f'envs/games/zelda_v0/levels/{f}'))

# os.rmdir(os.path.join(gym_gvgai.dir, 'envs/games/zelda_v0/levels'))

In [ ]:
from generator.env_gen_wrapper import GridGame

In [ ]:
from agent.base import simulate

In [ ]:
import numpy as np

In [ ]:
from baselines.common.vec_env.dummy_vec_env import DummyVecEnv

In [ ]:
from agent.NNagent import NNagent

In [ ]:
import torch

In [ ]:
popLim = 2
population = [None]*popLim

for i in range(len(population)):
    population[i] = NNagent(GridGame(game='zelda', 
                                     play_length=1000,
                                     path='./levels',
                                     lvl_name='start.txt', 
                                     mechanics=['1', '2', '3', '+', 'g', 'w'], # monsters, key, door, wall
                                     images=False
                                  )
                         )

In [ ]:
population.append(NNagent(GridGame(game='zelda', 
                                     play_length=1000,
                                     path='./levels',
                                     lvl_name='start.txt', 
                                     mechanics=['1', '2', '3', '+', 'g', 'w'], # monsters, key, door, wall
                                     images=False
                                  ), 
                          parent=torch.load("./25_gen_weights_5_5.pt")
                         )
                 )

In [ ]:
tile = population[0].reset()

In [ ]:
import time
start = time.time()
population[i].evaluate()
end = time.time() - start
print(end)

In [ ]:
# population[0].fitness(noisy=True, fn=show_state)

In [ ]:
# population[0].env._fit

In [ ]:
# def make():
#     def _make():
#         return GridGame(game='zelda', 
#                          play_length=250, 
#                          path=gym_gvgai.dir + '/envs/games/zelda_v0/', 
#                          lvl_name='zelda_lvl0.txt', 
#                          mechanics=['1', '2', '3', '+', 'g', 'w'], # monster, key, door, wall
#                                )
#     return _make

In [ ]:
# x = make()()

In [ ]:
# x.env.env.get_action_meanings()

In [ ]:
# def move(action):
#     a, b, c, d = x.step(action)
#     print(b)
#     plt.imshow(d['pic'])

In [ ]:
#  move(2)

In [ ]:
# env = [make(), make()]

In [ ]:
# from baselines.common.vec_env.shmem_vec_env import ShmemVecEnv

In [ ]:
# envs = ShmemVecEnv(env, context='fork')

In [ ]:
# envs.step_async([0, 0])

In [ ]:
# [pipe.poll() for pipe in envs.parent_pipes]

In [ ]:
# if np.all([pipe.poll() for pipe in envs.parent_pipes]):
#     outs = [pipe.recv() for pipe in envs.parent_pipes]

In [ ]:
# self.waiting_step = False
# obs, rews, dones, infos = zip(*outs)
# z = (envs._decode_obses(obs), np.array(rews), np.array(dones), infos)

In [ ]:
# z = envs2.step_wait()



- kill oldest.   

- Write mutate wrapper that lives at top of Pair that will create a new Pair obj  
- Copy the Agent, mutate the env.  

- Flip fiber bits in the Tensor NN input.  

----  
- Interestingly, POET has a default of `max_children=8` for each reporduce step. 

In [ ]:
population

In [ ]:
population[2].evaluate()

In [ ]:
population[0].env.fitness(population[2])

In [ ]:
from optimization.Optimizer import run_opt_n_steps

In [ ]:
for epochs in range(1):
    for i in range(len(population)):
        population[i].fitness(noisy=True)
    
    print('evaluated')
    for i in range(len(population)):
        for _ in range(2):
            population.append(population[i].mutate(0.7))
            
    print(f"parents + children = {len(population)} members")
    population = sorted(population, key=lambda x: x.env.id, reverse=True)[:10] 
    # sort by id, kill Pairs with smallest IDs. 
    print(f"new generation has {len(population)} members")
    
    opt_wrapper = [PyTorchObjective(agent) for agent in population]
    
    for each_pair in opt_wrapper:
        ans = run_opt_n_steps(each_pair, n=3, popsize=5)
        each_pair.update_nn(ans)

        
#     if epochs % steps_between_transfer == 0:
#         for i, pairs in enumerate(population):
#             best_nn = pairs.nn
#             for others in population:
                
#         pass

In [ ]:
population[0].env.fitness(population[2])